**Install required packages**

select GPU as device

In [9]:
!pip install torchsummary
from torchsummary import summary
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

**Define the Network**

Since parameters size is retricted, we can not suddenly expand and reduce channels in a layer, as it hurts learning weights. 
Instead start small (10 channels) and increase uniformly in baby steps.

Using padding in first two blocks, to preserve every pixel of information we got.

---



Block #1:

[Conv-> ReLU-> BatchNorm] -> [Conv-> ReLU-> BatchNorm] -> MaxPool -> Dropout


---


Block #2:

[Conv-> ReLU-> BatchNorm] -> [Conv-> ReLU-> BatchNorm] -> MaxPool -> Dropout


---


Block #3:

[Conv-> GAP]


---


Block #4:

[Flatten -> Log_SoftMax]


In [10]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,  10, 3, padding=1) #input:28X28X1, out:28X28X10,  kernel:3X3, RF: 3.
        self.conv2 = nn.Conv2d(10, 10, 3, padding=1) #input:28X28X10, out:28X28X10, kernel:3X3, RF: 5.
        self.pool1 = nn.MaxPool2d(2, 2)              #input:28X28X10, out:14X14X10, kernel:2X2, RF: 10
        self.conv3 = nn.Conv2d(10, 20, 3, padding=1) #input:14X14X10, out:14X14X20, kernel:3X3, RF: 12.
        self.conv4 = nn.Conv2d(20, 20, 3, padding=1) #input:14X14X20, out:14X14X20, kernel:3X3, RF: 14.
        self.pool2 = nn.MaxPool2d(2, 2)              #input:14X14X20, out:7X7X20,   kernel:2X2, RF: 28
        self.conv5 = nn.Conv2d(20, 30, 3, padding=0) #input:7X7X20, out:5X5X30,     kernel:3X3, RF: 30.
        self.conv6 = nn.Conv2d(30, 10, 3)            #input:5X5X30, out:3X3X10,     kernel:3X3, RF: 32.        
        self.avg_pool = nn.AvgPool2d(kernel_size=3, stride=3)#input:3X3X10, out:1X1X10, kernel:3X3

        self.batchNorm_1 = nn.BatchNorm2d(10)
        self.batchNorm_2 = nn.BatchNorm2d(10)
        self.batchNorm_3 = nn.BatchNorm2d(20)
        self.batchNorm_4 = nn.BatchNorm2d(20)
        self.batchNorm_5 = nn.BatchNorm2d(30)

        self.dropout = nn.Dropout(0.10) #10% performs well as found by trial

    def forward(self, x):
        x = self.pool1(self.batchNorm_2(F.relu(self.conv2(self.batchNorm_1(F.relu(self.conv1(x)))))))
        x = self.dropout(x)
        x = self.pool2(self.batchNorm_4(F.relu(self.conv4(self.batchNorm_3(F.relu(self.conv3(x)))))))
        x = self.dropout(x)
        x = self.conv6(self.batchNorm_5(F.relu(self.conv5(x))))          
        x = self.avg_pool(x) 
        x = x.view(-1, 10)
        return F.log_softmax(x)

**Instatiate Network**

Assign GPU to network model

Print the model



In [11]:
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 10, 28, 28]             100
       BatchNorm2d-2           [-1, 10, 28, 28]              20
            Conv2d-3           [-1, 10, 28, 28]             910
       BatchNorm2d-4           [-1, 10, 28, 28]              20
         MaxPool2d-5           [-1, 10, 14, 14]               0
           Dropout-6           [-1, 10, 14, 14]               0
            Conv2d-7           [-1, 20, 14, 14]           1,820
       BatchNorm2d-8           [-1, 20, 14, 14]              40
            Conv2d-9           [-1, 20, 14, 14]           3,620
      BatchNorm2d-10           [-1, 20, 14, 14]              40
        MaxPool2d-11             [-1, 20, 7, 7]               0
          Dropout-12             [-1, 20, 7, 7]               0
           Conv2d-13             [-1, 30, 5, 5]           5,430
      BatchNorm2d-14             [-1, 3

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


**Load Train and Test data**

set Block size.

Since we are using BatchNormalization, we should not normalize the data data while loading



In [12]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor()
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


**Train and Test Network Flow**

Print Logs

In [13]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')       


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

**Execute The Network**

Using SGD with learning rate = 0.01 and momentum.

Since majorority of learning is done by 10 epochs, after 10, reduce the learning rate (using a scheduler) so as to reduce overshooting of weights



In [14]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[10], gamma=0.1)
for epoch in range(1, 20):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0469, Accuracy: 9854/10000 (98.54%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0325, Accuracy: 9909/10000 (99.09%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0164, Accuracy: 9942/10000 (99.42%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0165, Accuracy: 9943/10000 (99.43%)



loss=0.014885426498949528 batch_id=468: 100%|██████████| 469/469 [00:09<00:00, 50.84it/s]



Test set: Average loss: 0.0168, Accuracy: 9943/10000 (99.43%)

